### Notebook to parse and clean the TCGA data from UCSC Xena

In [1]:
# libraries
import pandas as pd
import numpy as np
import re
import os

In [9]:
## directory and file names
dir_data = '../data'

file_tpm = dir_data + '/' + 'TcgaTargetGtex_rsem_gene_tpm.gz'
file_phenotype = dir_data + '/' + 'TcgaTargetGtex_phenotype.txt'
file_category = dir_data + '/' + 'TCGA_GTEX_category.txt'
file_probe = dir_data + '/' + 'gencode.v23.annotation.gene.probemap.tsv'


#### Read and summarize 

In [10]:
## tpm data
df_tpm = pd.read_csv(file_tpm, sep='\t', header=0, index_col=0, compression='gzip')
print(df_tpm.shape)

In [8]:
# get current working directory
cwd = os.getcwd()
print(cwd)


d:\Research\Omics\Xena\scripts
